# Pipeline de Limpeza e Tratamento de Dados
> **Objetivo:** Transformar dados brutos (Raw) em dados prontos para análise (Processed), garantindo a integridade e reprodutibilidade do projeto.

 =============================================================================
# criar o ambiente virtual e instalar as bibliotecas do requirements.md ==============================================================================
DQ Check

### 🛠️ Guia de Referência: Comandos de Limpeza (Sintaxe Pandas)

| Objetivo | Raciocínio de Negócio | Comando / Sintaxe |
| :--- | :--- | :--- |
| **Converter Data** | Datas como "texto" não permitem cálculos de tempo. | `pd.to_datetime(df['coluna'])` |
| **Limpar Nomes** | Colunas com espaços ou maiúsculas quebram o SQL. | `df.columns = [col.strip().lower() for col in df.columns]` |
| **Remover Duplicados** | Vendas duplicadas inflam o faturamento artificialmente. | `df.drop_duplicates(subset=['id_col'])` |
| **Tratar Nulos** | Valores vazios (NaN) resultam em erros em somas. | `df['coluna'].fillna(0)` |
| **Padronizar Texto** | Evitar que 'Paid' e 'paid' sejam contados como diferentes. | `df['coluna'].str.lower().str.strip()` |
| **Verificar Tipos** | Conferir se os dados estão prontos para análise. | `df.dtypes` ou `df.info()` |

Célula 1: Carregamento e Diagnóstico

In [28]:
import pandas as pd
import os

# Caminhos definitivos
PATH_RAW = '../data/raw/sales_raw_2025.csv'
PATH_SILVER = '../data/processed/sales_cleaned.csv'

# Carregamento
df = pd.read_csv(PATH_RAW)

print("Dados Brutos Carregados:")
df.info() # Mostra tipos e nulos de forma limpa

Dados Brutos Carregados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      5 non-null      int64  
 1   data    5 non-null      object 
 2   valor   4 non-null      float64
 3   status  5 non-null      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 292.0+ bytes


### 🛠️ Guia de Referência: Comandos de Limpeza

| Objetivo | Comando Pandas | O que ele faz? |
| :--- | :--- | :--- |
| **Ver nulos** | `df.isnull().sum()` | Conta quantos "buracos" existem em cada coluna. |
| **Tipagem** | `pd.to_datetime()` | Transforma texto em data real (essencial para MRR). |
| **Limpar Texto** | `.str.lower().str.strip()` | Deixa tudo minúsculo e tira espaços inúteis. |
| **Preencher Nulos** | `.fillna(valor)` | Troca o "vazio" por 0 ou por um texto padrão. |
| **Remover Lixo** | `.drop_duplicates()` | Deleta linhas repetidas para não inflar KPIs. |

2. Pipeline de Limpeza 



In [29]:
# 1. Limpeza de IDs e Duplicados
df = df.drop_duplicates(subset=['id'])

# 2. Padronização de Status e Datas
df['status'] = df['status'].str.strip().str.lower()
df['data'] = pd.to_datetime(df['data'], errors='coerce')

# 3. Tratamento de Valores
df['valor'] = df['valor'].fillna(0)

# 4. Exportação para a Camada Silver
os.makedirs('../data/processed', exist_ok=True)
df.to_csv(PATH_SILVER, index=False)

print(f"✅ Arquivo limpo salvo em: {PATH_SILVER}")
df.head()

✅ Arquivo limpo salvo em: ../data/processed/sales_cleaned.csv


,id,data,valor,status
0,1,2025-01-01,100.5,paid
1,2,2025-01-02,200.0,pending
3,3,2025-01-03,300.0,paid
4,4,NaT,0.0,cancelled


3. Validação Final e Salvamento

In [30]:
# Tabela de conferência rápida
check = pd.DataFrame({
    'Métrica': ['Nulos na Data', 'Duplicados', 'Status Padronizado'],
    'Resultado': [df['data'].isnull().sum(), df.duplicated().sum(), df['status'].unique()]
})

display(check)

# Salvar o resultado final
os.makedirs('../data/processed', exist_ok=True)
df.to_csv('../data/processed/sales_cleaned.csv', index=False)
print("Processo finalizado!")

,Métrica,Resultado
0,Nulos na Data,1
1,Duplicados,0
2,Status Padronizado,"[paid, pending, cancelled]"


Processo finalizado!
